In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [2]:
# Importa e inicia o findspark
import findspark
findspark.init()

In [3]:
# Importa todos os módulos necessários
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [4]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[7]") # usa a márquina local com 7 nucleos
                     .appName("Aceleração PySpark - Capgemini"))

In [5]:
# Cria o Schema de todos os datasets
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField('tailnum', StringType(), True),
    StructField('year', IntegerType(), True),
    StructField('type', StringType(), True),
    StructField('manufacturer', StringType(), True),
    StructField('model', StringType(), True),
    StructField('engines', IntegerType(), True),
    StructField('seats', IntegerType(), True),
    StructField('speed', IntegerType(), True),
    StructField('engine', StringType(), True)
])

schema_flights = StructType([
    StructField('year', IntegerType(), True),
    StructField('month', IntegerType(), True),
    StructField('day', IntegerType(), True),
    StructField('dep_time', StringType(), True),
    StructField('dep_delay', IntegerType(), True),
    StructField('arr_time', StringType(), True),
    StructField('arr_delay', IntegerType(), True),
    StructField('carrier', StringType(), True),
    StructField('tailnum', StringType(), True),
    StructField('flight', StringType(), True),
    StructField('origin', StringType(), True),
    StructField('dest', StringType(), True),
    StructField('air_time', IntegerType(), True),
    StructField('distance', IntegerType(), True),
    StructField('hour', IntegerType(), True),
    StructField('minute', IntegerType(), True)
])

In [6]:
# Cria os dataframes
df_airports = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("../data/airports.csv"))

df_planes = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("../data/planes.csv"))

df_flights = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("../data/flights.csv"))

In [7]:
df_airports.show(3)
df_planes.show(3)
df_flights.show(3)

+---+--------------------+---------+---------+----+---+---+
|faa|                name|      lat|      lon| alt| tz|dst|
+---+--------------------+---------+---------+----+---+---+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|
+---+--------------------+---------+---------+----+---+---+
only showing top 3 rows

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
+--

In [ ]:
# Função de regex
def regex_definition(element):
    return r'|'.join(map(lambda x : f'.*{x}*', element))

# -----------------------------------------------------------------------------------------------------------
# Transformação airports
# -----------------------------------------------------------------------------------------------------------
## 1.

In [ ]:
df_airports.filter(F.col('alt') < 0).show()

In [ ]:
df_airports = df_airports.withColumn('alt', 
                                     F.when(F.col('alt') < 0, 0)
                                      .otherwise(F.col('alt'))
                                    )

In [ ]:
df_airports.filter(F.col('alt') == 0).count()

In [ ]:
df_airports.groupBy(F.col('alt')).count().orderBy(F.col('alt')).show()

# -----------------------------------------------------------------------------------------------------------
## 2.

In [ ]:
df_airports.filter(F.col('tz').between(-7, -5)).show(10)

In [ ]:
df_airports = df_airports.withColumn('dst',
                                     F.when(F.col('tz').between(-7, -5), 'A')
                                      .otherwise(F.col('dst'))
                                    )

In [ ]:
df_airports.filter(F.col('tz').between(-7, -5)).show(10)
df_airports.groupBy("dst").count().show()

In [ ]:
df_airports.groupBy(F.col('dst')).count().show(999)

# -----------------------------------------------------------------------------------------------------------
## 3.

In [ ]:
df_airports.filter(F.col('dst') == 'U').show()

In [ ]:
df_airports = df_airports.withColumn('dst',
                                     F.when(F.col('dst') == 'U', 'A')
                                      .otherwise(F.col('dst'))
                                    )

In [ ]:
df_airports.filter(F.col('faa') == 'GCW').show()
df_airports.filter(F.col('dst') == 'A').count()
df_airports.groupBy("dst").count().show()